In [1]:
from langchain_community.document_loaders import UnstructuredHTMLLoader, BSHTMLLoader, TextLoader, JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import LlamaCpp
import pickle

In [2]:
persist_directory = "./embeddings"

with open(f"{persist_directory}/embeddings.pickle", 'rb') as handle:
    embeddings = pickle.load(handle)


In [3]:
vectordb = Chroma(
    "embeddings",
    embedding_function=embeddings,
    persist_directory=persist_directory,
    collection_metadata={"hnsw:space": "cosine"},
)

In [4]:
retriever = vectordb.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20},
)

In [5]:
from langchain_community.llms import LlamaCpp


llm_open = LlamaCpp(
    model_path="./models/llama-2-7b.Q3_K_M.gguf", # https://huggingface.co/TheBloke/Llama-2-7B-GGUF
    n_ctx=4096,  # 4096 for Llama, 32*1024 for Mistral
    n_gpu_layers=50,
    temperature=0.15,
    top_p=1,
    top_k=40,
    repeat_penalty=1.1,
    max_tokens=1024,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    stream=True,
)

/Users/patrick/.local/share/virtualenvs/capstone-s4bMLJni/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! stream is not default parameter.
                stream was transferred to model_kwargs.
                Please confirm that stream is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.b

In [6]:
def process_llm_response(llm_response):
    print("\n\nSources:")
    for i, source in enumerate(llm_response["source_documents"]):
        m = source.metadata
        print(f"{i + 1}. {m['course_title']} ({m['course_number']}): {m['heading']}")

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm_open,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

In [8]:
import pandas as pd

class ResponseFormatter:
    def __init__(self):
        self.responses = {
            "question": [],
            "contexts": [],
            "answer": [],
            "ground_truth": [],
        }

    def add_response(self, llm_response, ground_truth):
        self.responses["question"].append(llm_response["query"])
        self.responses["contexts"].append(
            [doc.page_content for doc in llm_response["source_documents"]]
        )
        self.responses["answer"].append(llm_response["result"])
        self.responses["ground_truth"].append(ground_truth)

    def get_responses(self):
        return self.responses
    
    def get_dataframe(self):
        pd.DataFrame(self.responses[["question", "answer", "ground_truth"]])

In [9]:
query = "Where can I find the MADS Incomplete Request Form?"

vectordb.similarity_search_with_score(query)

[(Document(page_content='Grades/Transcripts > Q: What is the process for requesting an incomplete?: To request a grade of incomplete ("I") in a course that has not been awarded a letter grade or course withdrawal (W), students must contact the MADS Academic Advising Team ([umsi.advising.mads@umich.edu](mailto:umsi.advising.mads@umich.edu)) as soon as possible, and no later than two business days (Eastern time) after the last day of the MADS session. Once contact is made, an academic advisor will partner with the student to create a plan for completion of the course, typically through placement and participation in a future session.\n\nThe MADS Academic Advising Team will notify the course instructor when a student’s request for an incomplete is approved so that an "I" can be submitted as the student’s final course grade.\n', metadata={'course_date': 'n/a', 'course_number': 'n/a', 'course_title': 'n/a', 'document': 'https://docs.google.com/document/d/1A3zdTF0AYQY_zzD2-OlpSHeDxnWqFVEhXl4

In [10]:
# query = "Tell me what I need to turn in for the capstone project."

llm_response = qa_chain(query)
process_llm_response(llm_response)

/Users/patrick/.local/share/virtualenvs/capstone-s4bMLJni/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...
 The MADS Incomplete Request Form can be found on the MADS website [HERE](https://www.si.umich.edu/current-students/mads-incomplete-request-form).



llama_print_timings:        load time =    9199.15 ms
llama_print_timings:      sample time =       9.55 ms /    48 runs   (    0.20 ms per token,  5028.28 tokens per second)
llama_print_timings: prompt eval time =   40431.27 ms /   879 tokens (   46.00 ms per token,    21.74 tokens per second)
llama_print_timings:        eval time =   10051.90 ms /    47 runs   (  213.87 ms per token,     4.68 tokens per second)
llama_print_timings:       total time =   51583.09 ms /   926 tokens



> Finished chain.


Sources:
1. n/a (n/a): Grades/Transcripts > Q: What is the process for requesting an incomplete?
2. n/a (n/a): Course Planning > Q: Where can I find the MADS Academic Calendar and Important Dates?
3. n/a (n/a): MADS Slack Policy, Standards, and Practices > MADS Slack Standards and Processes > Use of Slack Apps and Bots
4. n/a (n/a): Additional University Resources > Q: What mental health resources are available to MADS students?
5. n/a (n/a): General Academic Advising > Q: What is the best way to contact the MADS advising team?


In [11]:
process_llm_response(llm_response)



Sources:
1. n/a (n/a): Grades/Transcripts > Q: What is the process for requesting an incomplete?
2. n/a (n/a): Course Planning > Q: Where can I find the MADS Academic Calendar and Important Dates?
3. n/a (n/a): MADS Slack Policy, Standards, and Practices > MADS Slack Standards and Processes > Use of Slack Apps and Bots
4. n/a (n/a): Additional University Resources > Q: What mental health resources are available to MADS students?
5. n/a (n/a): General Academic Advising > Q: What is the best way to contact the MADS advising team?


In [12]:
import pandas as pd

df = pd.read_csv("./evaluation_set.csv")

for _, row in df.iterrows():
    print(row["question"])

Which class involves time series analysis?
Who teaches the SQL and Databases class?
What are the prerequisites for Data Science for Social Good?
When are the office hours for the Math Methods course?
Are there any weekly readings for Milestone II?
What are the outcomes of Qualitative Inquiry?
What textbook is required for SIADS 505?
What textbook is required for Data Manipulation?
Which week of unsupervised learning covers DBSCAN?
How many credits are required to complete the MADS program?
How long do students have to complete the MADS program start to finish?


In [13]:
formatter = ResponseFormatter()

In [14]:
for _, row in df.iterrows():
    llm_response = qa_chain(row["question"])
    process_llm_response(llm_response)

    formatter.add_response(llm_response, row["ground_truth"])

    # Save after each response
    with open(f"./evaluation_set_formatter.pickle", 'wb') as handle:
        pickle.dump(formatter, handle)

    with open(f"./evaluation_set_responses.pickle", 'wb') as handle:
        pickle.dump(formatter.get_responses(), handle)



> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit


 Data Manipulation and Visual Exploration of Data both involve time series analysis.

Data Manipulation (SIADS 511), Course Schedule: This course begins on Tuesday November 14, 2023, and ends on Monday, December 4, 2023.

Assignments should be submitted by 11:59 pm on the date they are due:

- Assignment 1: Nov 14, 2023
- Assignment 2: Nov 21, 2023
- Assignment 3: Nov 28, 2023
- Assignment 4 & 5: Dec 4, 2023

Visual Exploration of Data (SIADS 521), Course Schedule: This course begins on Tuesday November 14, 2023, and ends on Monday, December 4, 2023.

Assignments should be submitted by 11:59 pm on the date they are due:

- Assignment 1: Nov 14, 2023
- Assignment 2: Nov 21, 2023
- Assignment 3: Nov 28, 2023
- Assignment 4 & 5: Dec 4, 2023

Communicating Data Science Results (SIADS 523), Course Schedule: This course begins on Tuesday November 14, 2023, and ends on Monday, December 4, 2023.

Assignments should be submitted by 11:59 pm on the date they are due:

- Assignment 1: Nov 14, 202


llama_print_timings:        load time =    9199.15 ms
llama_print_timings:      sample time =     136.91 ms /   759 runs   (    0.18 ms per token,  5543.91 tokens per second)
llama_print_timings: prompt eval time =   28869.81 ms /   790 tokens (   36.54 ms per token,    27.36 tokens per second)
llama_print_timings:        eval time =   90460.31 ms /   758 runs   (  119.34 ms per token,     8.38 tokens per second)
llama_print_timings:       total time =  128162.88 ms /  1548 tokens



> Finished chain.


Sources:
1. Data Mining II (SIADS 632): Learning Outcomes
2. n/a (n/a): General Academic Advising > Q: Can you advise me on which courses are the heaviest workload, the most difficult, etc?
3. Health Analytics (SIADS 681): Weekly Readings or Textbook Information
4. Visual Exploration of Data (SIADS 521): Syllabus Course Overview And Prerequisites
5. Communicating Data Science Results (SIADS 523): Course Schedule


> Entering new RetrievalQA chain...


...


> Finished chain.


Sources:
1. n/a (n/a): Grades/Transcripts > Q: What is the minimum GPA requirement for MADS students?
2. n/a (n/a): Course Planning > Q: What is the recommended number of credits students should take?
3. n/a (n/a): Course Planning > Q: When will courses be offered next?
4. Milestone II (SIADS 696): Letter Grades, Course Grades
5. Unsupervised Learning (SIADS 543): Grading And Course Checklist


> Entering new RetrievalQA chain...


Llama.generate: prefix-match hit


 Per the [MADS Student Handbook](https://docs.google.com/document/d/1YEOcpdONdme5kmpNEnZpdbJeVFhEIw1pS0wq16QdH1I/edit?usp=sharing), "students are expected to complete all coursework within five years from the date of first enrollment in the program. Students exceeding this time limit must file a petition for modification or waiver of policy. Petitions must describe explicitly the amount of work remaining and a timeline for completion. Students must be making good progress and have good academic standing to receive an extension. A student who does not complete degree requirements within five years and does not petition for an extension may be withdrawn from the program and required to apply for readmission".


llama_print_timings:        load time =    9199.15 ms
llama_print_timings:      sample time =      85.27 ms /   182 runs   (    0.47 ms per token,  2134.32 tokens per second)
llama_print_timings: prompt eval time =   43382.21 ms /   957 tokens (   45.33 ms per token,    22.06 tokens per second)
llama_print_timings:        eval time =   37966.21 ms /   181 runs   (  209.76 ms per token,     4.77 tokens per second)
llama_print_timings:       total time =   89992.42 ms /  1138 tokens



> Finished chain.


Sources:
1. n/a (n/a): General Academic Advising > Q: How long do I have to complete the program?
2. n/a (n/a): Course Planning > Q: How are the MADS courses/semesters set up?
3. n/a (n/a): Grades/Transcripts > Q: What is the minimum GPA requirement for MADS students?
4. Milestone II (SIADS 696): Letter Grades, Course Grades
5. n/a (n/a): MADS Slack Policy, Standards, and Practices > MADS Slack Standards and Processes > Leaving Course Channels
